### pycocotools를 이용한 COCO 데이터 액세스와 Segmentation Masking 시각화
* pycocotools 사용부분은 https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb 를 참조하였음. 
* pycocotools를 활용하여 coco dataset을 탐색
* pycocotools는 colab에서 이미 설치됨. 
* pycocotools 설치는 pip로 설치 시 오류가 발생하기 쉬우므로 conda 로 설치하거나 아예 git에서 소스코드를 다운로드 받아서 설치. 


In [ ]:
from pycocotools.coco import COCO

import numpy as np

### COCO 데이터 세트 Annotation 다운로드 
1. COCO 데이터 세트는 아래에서 다운로드 가능
http://cocodataset.org/#download

2. 2017 Train/Val annotation 압축 파일을 다운로드 한 뒤, /content/data에 압축 해제  
2017년 Train image 파일 다운로드: wget http://images.cocodataset.org/zips/train2017.zip  
2017년 Val image 파일 다운로드: wget http://images.cocodataset.org/zips/val2017.zip  
2017년 Train/Val annoation 파일 다운로드: http://images.cocodataset.org/annotations/annotations_trainval2017.zip  


In [ ]:
!mkdir -p /content/data

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip -d /content/data

### COCO Annotation 정보 확인하기

In [ ]:
dataDir='/content/data'
dataType='val2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)
print(annFile)

In [ ]:
!ls -lia /content/data/annotations/instances_val2017.json

In [ ]:
!sudo apt-get install jq

In [ ]:
!jq . /content/data/annotations/instances_val2017.json > output.json 

In [ ]:
!head -200 output.json

In [ ]:
!tail -600 output.json

In [ ]:
!grep -n 'annotations' output.json

In [ ]:
!head -50300 output.json | tail -300 

### COCO API를 활용하기 위해 annotation 파일을 COCO 객체로 로드하기

In [ ]:
# annotation 파일을 COCO객체로 로드하면 다양한 COCO객체의 API들을 이용하여 COCO DATASET 활용 가능
from pycocotools.coco import COCO

coco=COCO(annFile)

#### Cagory 정보를 가져 오기

In [ ]:
# getCatIds()는 COCO Dataset의 category id를 리스트로 반환
print(coco.getCatIds())

In [ ]:
# loadCats()는 category id 리스트를 입력받아 category들에 대한 세부 정보를 여러개의 딕셔너리를 개별 원소를 가지는 리스트로 반환
cats = coco.loadCats(coco.getCatIds())
cats

In [ ]:
# COCO Category와 Super Category 출력
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

### 지정된 이미지를 데이터 세트에서 로드하기 

In [ ]:
catIds = coco.getCatIds(catNms=['person','dog','skateboard']);
print(catIds)
# oco.getImgIds(catIds=catIds)는 해당 catogory id별로 한개의 image id을 임의로 출력
imgIds = coco.getImgIds(catIds=catIds )
print(imgIds)

In [ ]:
#loadImgs()는 인자로 들어온 image id에 대한 메타 정보를 딕셔너리를 개별 원소로 가지는 리스트로 반환
img = coco.loadImgs(324158)
print(img)

# 전체 리스트는 필요 없고 내부 딕셔너리만 필요하므로 [0]으로 내부 딕셔너리 추출 
print("\n내부 딕셔너리 파일 메타정보 추출")
img = coco.loadImgs(324158)[0]
print(img)

#### COCO 이미지를 다운로드 후 시각화

In [ ]:
coco_url = img['coco_url']
print(coco_url)

In [ ]:
import urllib.request

def download_image(url, target_path):
  urllib.request.urlretrieve(url, target_path) 

download_image(img['coco_url'], '/content/data/' + img['file_name']) 


In [ ]:
import cv2
import matplotlib.pyplot as plt
import pylab
%matplotlib inline

file_path = '/content/data/' + img['file_name']

image_array = cv2.imread(file_path)
image_array = cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 14))
plt.axis('off')
plt.imshow(image_array)
plt.show()

### Instance Segmentation 시각화 - COCO API 활용한 시각화
* getAnnIds()로 특정 image에 해당하는 annotation id를 가져온 후에 이 id를 loadAnns()로 입력하여 해당 이미지의 모든 annotation 정보를 가져옴. 
* segmentation 정보는 polygon 형태로 되어 있음. 
* annotation 정보를 coco.showAnns(anns)에 입력하여 instance segmentation 시각화 수행. 

In [ ]:
# 해당 image의 annotation을 가져오기 위해서 getAnnIds() 를 이용함. 인자로 image의 id(파일명이 아님)와 category id를 입력
# 하나의 image는 segmentation별로 여러개의 annotation을 가질 수 있음
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
annIds

In [ ]:
# loadAnns()에 annotation id를 리스트로 입력하면 annotation 정보들을 반환함. 
anns = coco.loadAnns(annIds)
anns

In [ ]:
# showAnns( )는 annotation 정보들을 입력 받아서 Visualization 시켜줌. 단 먼저 matplotlib 객체로 원본 이미지가 먼저 로드되어 있어야 함. 
plt.figure(figsize=(12, 14))
plt.imshow(image_array)
plt.axis('off')

coco.showAnns(anns)

### Polygon Annotation 정보를 이용하여 Instance Segmentation 시각화
* Polygon 정보를 직접 추출하여 이를 시각화
* polygon 정보를 coco API를 활용하여 masking 형태로 변환 후 해당 masking을 원본 이미지에 적용하여 시각화.  

In [ ]:
len(anns), type(anns[0])

In [ ]:
# 앞에서 추출한 annotations 정보를 가지는 anns list의 의 3번째 값(인덱스 2)에서 segmentation polygon 값만 추출
print(anns[2]['segmentation'])
ann_2_seg = anns[2]['segmentation'][0]
print(type(ann_2_seg), len(ann_2_seg))
print(ann_2_seg)

In [ ]:
 # x,y 좌표값이 연이어 되어 있는 list형 polygon segmentation 정보를 x,y 쌍 형태로 변환. 
 polygon_x = [x for index, x in enumerate(ann_2_seg) if index % 2 == 0]
 polygon_y = [x for index, x in enumerate(ann_2_seg) if index % 2 == 1]
 print('polygon_x:', polygon_x)
 print('polygon_y:', polygon_y)
 polygon_xy = [[x, y] for x, y in zip(polygon_x, polygon_y)]
 print('polygon_xy:', polygon_xy)

In [ ]:
import numpy as np

# opencv의 polylines를 이용하여 변환된 polygon 좌표 적용하여 instance segmentation 외곽선 시각화
green_color = (0, 255, 0)

draw_img = image_array.copy()
polygon_xy = np.array(polygon_xy, np.int32)
draw_img = cv2.polylines(draw_img, [polygon_xy], True, (0, 255, 0))

plt.figure(figsize=(12, 14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
# opencv의 fillPoly를 이용하여 변환된 polygon 좌표 적용하여 instance segmentation 내부 시각화
green_color = (0, 255, 0)

draw_img = image_array.copy()
polygon_xy = np.array(polygon_xy, np.int32)
draw_img = cv2.fillPoly(draw_img, [polygon_xy], (0, 255, 0))

plt.figure(figsize=(12, 14))
plt.imshow(draw_img)
plt.axis('off')

In [ ]:
# coco api의 annToMask()를 이용하여 polygon을 mask 형태로 변환
mask = coco.annToMask(anns[2])
print('image shape:', image_array.shape, 'mask shape:', mask.shape)
print('0보다 큰 값이 있는 mask shape:', mask[mask > 0].shape, '0이 있는 mask shape:', mask[mask == 0].shape)
print(mask[mask > 0])

In [ ]:
plt.figure(figsize=(12, 14))
plt.imshow(mask)
plt.axis('off')

### Polygon 정보를 mask로 변환하여 Instance Segmentation 시각화 
* cv2.fillPoly()로 polygon을 mask형태로 변환.
* layer별 masking 영역에서 0을 제외한 pixel 영역을 layer별 시각화

In [ ]:
np.zeros(image_array.shape[0:2])

In [ ]:
## polygon 정보를 mask 정보로 변환. 
zero_mask = np.zeros(image_array.shape[0:2]) # mask 정보는 2차원 선호
masked_polygon = cv2.fillPoly(zero_mask, [polygon_xy], 1)
print('zero_mask shape:', zero_mask.shape, 'mask shape:', masked_polygon.shape)
print('masked_polygon 0보다 큰 값:', masked_polygon[masked_polygon > 0])
print(masked_polygon[masked_polygon > 0].shape, masked_polygon[masked_polygon == 0].shape)

plt.figure(figsize=(12, 14))
plt.imshow(masked_polygon)
plt.axis('off')

In [ ]:
# 보통 bool 값으로 masking 정보 제공. 
masked_bool = masked_polygon.astype(bool)
print('masked_bool shape:', masked_bool.shape)
print(masked_bool[masked_bool==1].shape, masked_bool[masked_bool==0].shape)
print(masked_bool[masked_bool==1])

In [ ]:
# mask 값에 따라 channel 별로 alpha값을 감안하여 색상 적용
def apply_mask_01(image, mask, color, alpha=0.5):
  for c in range(3):
    image[:, :, c] = np.where(mask == 1,
                              image[:, :, c] *
                              (1 - alpha) + alpha * color[c] * 255,
                              image[:, :, c])
  return image

In [ ]:
draw_img = image_array.copy()
# masking 정보는 2차원 bool array로 전달. 
masked_image = apply_mask_01(draw_img, masked_bool, (0, 255, 0), alpha=0.6)

plt.figure(figsize=(12, 14))
plt.imshow(masked_image)
plt.axis('off')

In [ ]:
def apply_mask_02(image, mask, color, alpha=0.5):
  """Apply the given mask to the image.
  """
  image = np.where(mask == 1, color, image)
  return image

In [ ]:
 np.stack([masked_bool, masked_bool, masked_bool], axis=2).shape

In [ ]:
draw_img = image_array.copy()
stacked_mask = np.stack([masked_bool, masked_bool, masked_bool], axis=2)
masked_image = apply_mask_02(draw_img, stacked_mask, (0, 255, 0))
plt.figure(figsize=(12, 14))
plt.imshow(masked_image)
plt.axis('off')

### bitwise_and 연산으로 오브젝트 영역을 제외하고 나머지 영역은 모두 0 처리하기

In [ ]:
# True는 255로, False는 0으로 변환. 
s_mask_int = (masked_bool * 255).astype("uint8")
draw_img = image_array.copy()
# opencv의 bitwise_and()로 masking 255 영역만 object 나타내고, 0 영역은 검은색으로 시각화  
only_mask_image = cv2.bitwise_and(draw_img, draw_img, mask=s_mask_int)
plt.figure(figsize=(12, 14))
plt.imshow(only_mask_image)
plt.axis('off')

In [ ]:
masked_image.shape

### Object segmentation의 외곽선 그리기
* opencv의 findContours()와 drawContours()를 이용하여 외곽선을 그림. 
* 2차원 masking bool array를 0또는 255 숫자로 변경한 후 findContours()를 적용. 
* findContours() 설명은 https://bkshin.tistory.com/entry/OpenCV-22-%EC%BB%A8%ED%88%AC%EC%96%B4Contour 참조

In [ ]:
draw_img = image_array.copy()

s_mask_int = (masked_bool * 255).astype("uint8")
contours, hierarchy = cv2.findContours(s_mask_int, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
masked_image = cv2.drawContours(draw_img, contours, -1, (255, 0, 0), 1, cv2.LINE_8, hierarchy, 100)

plt.figure(figsize=(12, 14))
plt.imshow(masked_image)
plt.axis('off')

### Segmentation 코드 함수화

In [ ]:
def get_polygon_xy(ann_seg):
  polygon_x = [x for index, x in enumerate(ann_seg) if index % 2 == 0]
  polygon_y = [x for index, x in enumerate(ann_seg) if index % 2 == 1]
  polygon_xy = [[x, y] for x, y in zip(polygon_x, polygon_y)]
  polygon_xy = np.array(polygon_xy, np.int32)
  return polygon_xy

def get_mask(image_array_shape, polygon_xy):
  mask = np.zeros(image_array_shape)
  masked_polygon = cv2.fillPoly(mask, [polygon_xy], 1)
  
  return masked_polygon

def apply_mask(image, mask, color, alpha=0.5):
  for c in range(3):
    image[:, :, c] = np.where(mask == 1,
                              image[:, :, c] *
                              (1 - alpha) + alpha * color[c] * 255,
                              image[:, :, c])
  return image

# ann_seg_list에 있는 object들의 segmentation에 따라 instance segmentation 시각화. 
def draw_segment(image_array, ann_seg_list, color_list, alpha):
  draw_image = image_array.copy()
  mask_array_shape = draw_image.shape[0:2]

  # list형태로 입력된 segmentation 정보들을 각각 시각화
  for index, ann_seg in enumerate(ann_seg_list):
    # polygon 좌표로 변환. 
    polygon_xy = get_polygon_xy(ann_seg)
    # mask 정보 변환
    masked_polygon = get_mask(mask_array_shape, polygon_xy)

    # segmentation color와 외곽선용 color 선택 
    color_object = color_list[np.random.randint(len(color_list))]
    color_contour = color_list[np.random.randint(len(color_list))]
    # masking 적용. 
    masked_image = apply_mask(draw_image, masked_polygon, color_object, alpha=0.6)
    # 외곽선 적용. 
    s_mask_int = (masked_polygon*255).astype("uint8")
    contours, hierarchy = cv2.findContours(s_mask_int, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    masked_image = cv2.drawContours(masked_image, contours, -1, color_contour, 1, cv2.LINE_8, hierarchy, 100)

  return masked_image


In [ ]:
color_list = [
              (0, 255, 0),
              (255, 0, 0),
              (0, 0, 255)
]
anns = coco.loadAnns(annIds)
ann_seg_list = [ann['segmentation'][0] for ann in anns]
print(ann_seg_list)

In [ ]:
masked_image = draw_segment(image_array, ann_seg_list, color_list, alpha=0.6)
plt.figure(figsize=(12, 14))
plt.imshow(masked_image)
plt.axis('off')